In [50]:
from itertools import groupby
from operator import itemgetter

import pandas as pd
import numpy as np
from pyproj import Proj, Transformer
from tqdm import tqdm
from shapely.geometry import Point, LineString
import datetime

import folium
from keplergl import KeplerGl

In [51]:
# Importing data
ngsim = pd.read_csv("Next_Generation_Simulation__NGSIM__Vehicle_Trajectories_and_Supporting_Data.csv")

len(ngsim)

11850526

In [ ]:
data = pd.DataFrame()

for name in ngsim.Location.unique():
    ngsim[ngsim.Location == name]

In [52]:
# Conversion of NAD83 to WGS84

locdict = {"i-80": Transformer.from_crs(2227, 4326, always_xy=True), 
           "us-101": Transformer.from_crs(2229, 4326, always_xy=True), 
           "lankershim": Transformer.from_crs(2229, 4326, always_xy=True), 
           "peachtree": Transformer.from_crs(2240, 4326, always_xy=True)}

coords = []

# Coordinate conversion
for i in tqdm(range(len(ngsim))):
    coord = locdict[ngsim["Location"].iloc[i]].transform(ngsim["Global_X"].iloc[i], ngsim["Global_Y"].iloc[i])
    coords.append(coord[::-1])
    
ngsim["Coordinates"] = coords

ngsim.head()

100%|████████████████████████████| 11850526/11850526 [03:08<00:00, 62765.82it/s]


,Vehicle_ID,Frame_ID,Total_Frames,Global_Time,Local_X,Local_Y,Global_X,Global_Y,v_length,v_Width,...,Int_ID,Section_ID,Direction,Movement,Preceding,Following,Space_Headway,Time_Headway,Location,Coordinates
0,515,2330,1123,1118848075000,30.034,188.062,6451203.729,1873252.549,13.0,6.9,...,NaN,NaN,NaN,NaN,500,523,119.10,5.11,us-101,"(34.13923367949842, -118.365050201637)"
1,515,2330,1123,1118848075000,30.034,188.062,6451203.729,1873252.549,13.0,6.9,...,NaN,NaN,NaN,NaN,500,523,119.10,5.11,us-101,"(34.13923367949842, -118.365050201637)"
2,2224,6548,1902,1113437421700,41.429,472.901,6042814.264,2133542.012,14.3,6.9,...,NaN,NaN,NaN,NaN,2208,2211,53.34,2.01,i-80,"(37.84064737954841, -122.29690230242794)"
3,2127,6459,567,1118847624800,19.632,1775.614,6452425.122,1872172.475,13.5,6.9,...,NaN,NaN,NaN,NaN,2124,2132,48.92,1.30,us-101,"(34.13627791851052, -118.36100104317349)"
4,1033,4827,592,1118848324700,6.202,1701.144,6452347.673,1872258.452,13.5,4.4,...,NaN,NaN,NaN,NaN,1029,1040,38.81,0.92,us-101,"(34.13651340589235, -118.36125800107918)"


In [53]:
ngsim['lat'] = ngsim['Coordinates'].apply(lambda x: x[0])
ngsim['lon'] = ngsim['Coordinates'].apply(lambda x: x[1])

In [55]:
m = KeplerGl()
m.add_data(ngsim[ngsim.Vehicle_ID == 515][['lat', 'lon']])
m

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'unnamed': {'index': [0, 1, 1098, 1099, 1428, 5823, 5990, 7125, 7425, 9457, 9508, 9509, 11683, …

In [34]:
df = ngsim[ngsim['Vehicle_ID'] == 515]
df = df[df['Location'] == 'us-101'].sort_values(by='Frame_ID')

In [46]:
df = df.drop_duplicates()

In [41]:
sum([402, 1123,  692])

2217

In [48]:
df[df['Total_Frames'] == 1123]

,Vehicle_ID,Frame_ID,Total_Frames,Global_Time,Local_X,Local_Y,Global_X,Global_Y,v_length,v_Width,...,Direction,Movement,Preceding,Following,Space_Headway,Time_Headway,Location,Coordinates,lat,lon
1079722,515,2286,1123,1118848070600,29.157,74.280,6451127.282,1873337.504,13.0,6.9,...,NaN,NaN,486,0,173.22,6.24,us-101,"(34.1394663599912, -118.36530385644416)",34.139466,-118.365304
4867227,515,2287,1123,1118848070700,29.167,77.055,6451129.129,1873335.433,13.0,6.9,...,NaN,NaN,486,0,172.95,6.23,us-101,"(34.1394606876371, -118.36529772777156)",34.139461,-118.365298
4584005,515,2288,1123,1118848070800,29.178,79.830,6451130.976,1873333.361,13.0,6.9,...,NaN,NaN,486,0,172.67,6.22,us-101,"(34.139455012534825, -118.36529159908777)",34.139455,-118.365292
5082347,515,2289,1123,1118848070900,29.189,82.605,6451132.822,1873331.290,13.0,6.9,...,NaN,NaN,486,0,172.39,6.21,us-101,"(34.13944934017009, -118.36528547372156)",34.139449,-118.365285
1909567,515,2290,1123,1118848071000,29.200,85.380,6451134.669,1873329.218,13.0,6.9,...,NaN,NaN,486,0,172.12,6.20,us-101,"(34.13944366506721, -118.36527934503944)",34.139444,-118.365279
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3250336,515,3404,1123,1118848182400,30.384,2155.761,6452679.454,1871945.238,13.0,6.9,...,NaN,NaN,0,523,0.00,0.00,us-101,"(34.1356560121152, -118.36015789089268)",34.135656,-118.360158
8255871,515,3405,1123,1118848182500,30.382,2158.057,6452681.234,1871943.822,13.0,6.9,...,NaN,NaN,0,523,0.00,0.00,us-101,"(34.13565213868347, -118.36015199200962)",34.135652,-118.360152
1074958,515,3406,1123,1118848182600,30.380,2160.366,6452683.072,1871942.360,13.0,6.9,...,NaN,NaN,0,523,0.00,0.00,us-101,"(34.13564813942131, -118.360145900918)",34.135648,-118.360146
1243361,515,3407,1123,1118848182700,30.379,2162.367,6452684.637,1871941.114,13.0,6.9,...,NaN,NaN,0,523,0.00,0.00,us-101,"(34.13564473100694, -118.36014071452921)",34.135645,-118.360141


In [49]:
m = KeplerGl()
m.add_data(df[df['Total_Frames'] == 1123][['lat', 'lon']])
m

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'unnamed': {'index': [1079722, 4867227, 4584005, 5082347, 1909567, 4788115, 1247354, 7791477, 5…

In [27]:
ngsim['Global_Time'][0]

1118848075000

In [23]:
ngsim['Local_Time'] = ngsim['Global_Time'].apply(lambda x: datetime.datetime.fromtimestamp(x))

ValueError: year 37424 is out of range

In [7]:
ngsim[ngsim['Vehicle_ID'] == 515].sort_values(by='Frame_ID')

,Vehicle_ID,Frame_ID,Total_Frames,Global_Time,Local_X,Local_Y,Global_X,Global_Y,v_length,v_Width,...,D_Zone,Int_ID,Section_ID,Direction,Movement,Preceding,Following,Space_Headway,Time_Headway,Location
9805175,515,648,346,1113436831700,12.044,29.691,6042839.675,2133098.719,8.9,3.7,...,NaN,NaN,NaN,NaN,NaN,552,0,68.88,1.61,i-80
10498955,515,649,346,1113436831800,11.992,33.966,6042839.075,2133102.952,8.9,3.7,...,NaN,NaN,NaN,NaN,NaN,552,0,67.58,1.58,i-80
8881810,515,650,346,1113436831900,11.940,38.241,6042838.476,2133107.185,8.9,3.7,...,NaN,NaN,NaN,NaN,NaN,534,0,32.32,0.76,i-80
11848788,515,651,346,1113436832000,11.888,42.517,6042837.876,2133111.419,8.9,3.7,...,NaN,NaN,NaN,NaN,NaN,534,0,31.04,0.73,i-80
8989224,515,652,346,1113436832100,11.832,46.825,6042837.277,2133115.652,8.9,3.7,...,NaN,NaN,NaN,NaN,NaN,534,0,30.24,0.71,i-80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7639765,515,4135,964,1118936093400,11.122,1591.786,6452447.472,1874270.562,12.9,6.8,...,208.0,4.0,0.0,2.0,1.0,0,556,0.00,0.00,lankershim
8462312,515,4136,964,1118936093500,10.487,1597.136,6452446.168,1874276.749,12.9,6.8,...,208.0,0.0,5.0,2.0,1.0,0,556,0.00,0.00,lankershim
8206287,515,4137,964,1118936093600,9.745,1600.384,6452444.968,1874281.656,12.9,6.8,...,208.0,0.0,5.0,2.0,1.0,0,556,0.00,0.00,lankershim
5395074,515,4138,964,1118936093700,10.535,1609.512,6452443.031,1874290.611,12.9,6.8,...,208.0,0.0,5.0,2.0,1.0,0,556,0.00,0.00,lankershim


In [3]:
ngsim.head()

,Vehicle_ID,Frame_ID,Total_Frames,Global_Time,Local_X,Local_Y,Global_X,Global_Y,v_length,v_Width,...,D_Zone,Int_ID,Section_ID,Direction,Movement,Preceding,Following,Space_Headway,Time_Headway,Location
0,515,2330,1123,1118848075000,30.034,188.062,6451203.729,1873252.549,13.0,6.9,...,NaN,NaN,NaN,NaN,NaN,500,523,119.10,5.11,us-101
1,515,2330,1123,1118848075000,30.034,188.062,6451203.729,1873252.549,13.0,6.9,...,NaN,NaN,NaN,NaN,NaN,500,523,119.10,5.11,us-101
2,2224,6548,1902,1113437421700,41.429,472.901,6042814.264,2133542.012,14.3,6.9,...,NaN,NaN,NaN,NaN,NaN,2208,2211,53.34,2.01,i-80
3,2127,6459,567,1118847624800,19.632,1775.614,6452425.122,1872172.475,13.5,6.9,...,NaN,NaN,NaN,NaN,NaN,2124,2132,48.92,1.30,us-101
4,1033,4827,592,1118848324700,6.202,1701.144,6452347.673,1872258.452,13.5,4.4,...,NaN,NaN,NaN,NaN,NaN,1029,1040,38.81,0.92,us-101


In [94]:
# Dropping duplicates of same vehicle in same frame
ngsim=ngsim.drop_duplicates(subset = ["Vehicle_ID", "Frame_ID"])

In [99]:
# Example Vehicle
exvehicle = ngsim[(ngsim["Vehicle_ID"] == 515) & (ngsim["Location"] == "us-101")]

In [100]:
exvehicle.head()

,Vehicle_ID,Frame_ID,Total_Frames,Global_Time,Local_X,Local_Y,Global_X,Global_Y,v_length,v_Width,...,Int_ID,Section_ID,Direction,Movement,Preceding,Following,Space_Headway,Time_Headway,Location,Coordinates
0,515,2330,1123,1118848075000,30.034,188.062,6451203.729,1873252.549,13.0,6.9,...,NaN,NaN,NaN,NaN,500,523,119.10,5.11,us-101,"(34.13923367949842, -118.365050201637)"
1098,515,2331,1123,1118848075100,30.063,190.410,6451205.318,1873250.798,13.0,6.9,...,NaN,NaN,NaN,NaN,500,523,119.63,5.10,us-101,"(34.13922888386905, -118.36504492945205)"
7425,515,2933,692,1118849050300,43.024,1277.242,6452008.327,1872506.000,16.5,6.9,...,NaN,NaN,NaN,NaN,508,526,96.14,1.94,us-101,"(34.1371902743557, -118.36238235100879)"
9457,515,1411,402,1118847120000,31.385,642.578,6451564.527,1872909.752,19.0,8.4,...,NaN,NaN,NaN,NaN,502,518,174.33,3.52,us-101,"(34.13829531917491, -118.36385377132908)"
9508,515,2373,1123,1118848079300,28.549,281.971,6451270.837,1873184.648,13.0,6.9,...,NaN,NaN,NaN,NaN,500,523,146.89,7.38,us-101,"(34.13904776718388, -118.3648276153141)"


In [130]:
# Grouping by appearance - consecutive frames.

vehicle_t = sorted(list(exvehicle["Frame_ID"]))
groups = []

for k, g in groupby(enumerate(vehicle_t), lambda ix:ix[0]-ix[1]):
    groups.append(list(map(itemgetter(1), g)))

In [131]:
# By appearance. This is the appearance from 2286 - 2755.

def appearance(i):
    return exvehicle[exvehicle["Frame_ID"].isin(groups[i])]

appearance(1)

,Vehicle_ID,Frame_ID,Total_Frames,Global_Time,Local_X,Local_Y,Global_X,Global_Y,v_length,v_Width,...,Int_ID,Section_ID,Direction,Movement,Preceding,Following,Space_Headway,Time_Headway,Location,Coordinates
0,515,2330,1123,1118848075000,30.034,188.062,6451203.729,1873252.549,13.0,6.9,...,NaN,NaN,NaN,NaN,500,523,119.10,5.11,us-101,"(34.13923367949842, -118.365050201637)"
1098,515,2331,1123,1118848075100,30.063,190.410,6451205.318,1873250.798,13.0,6.9,...,NaN,NaN,NaN,NaN,500,523,119.63,5.10,us-101,"(34.13922888386905, -118.36504492945205)"
9508,515,2373,1123,1118848079300,28.549,281.971,6451270.837,1873184.648,13.0,6.9,...,NaN,NaN,NaN,NaN,500,523,146.89,7.38,us-101,"(34.13904776718388, -118.3648276153141)"
70508,515,2553,692,1118849012300,38.604,116.364,6451152.060,1873295.790,16.5,6.9,...,NaN,NaN,NaN,NaN,510,0,56.23,3.76,us-101,"(34.13935198356104, -118.36522147145186)"
76705,515,2537,1123,1118848095700,29.025,730.234,6451601.689,1872880.267,13.0,6.9,...,NaN,NaN,NaN,NaN,500,523,148.79,5.44,us-101,"(34.13821466837355, -118.36373061080882)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8610113,515,2465,1123,1118848088500,29.164,510.936,6451437.491,1873026.679,13.0,6.9,...,NaN,NaN,NaN,NaN,500,523,181.91,6.06,us-101,"(34.138615352794226, -118.36427498297913)"
8612857,515,2435,1123,1118848085500,29.770,425.598,6451374.327,1873084.065,13.0,6.9,...,NaN,NaN,NaN,NaN,500,523,181.96,6.44,us-101,"(34.13877241177237, -118.36448440724413)"
8620387,515,2469,1123,1118848088900,29.420,522.932,6451446.139,1873018.360,13.0,6.9,...,NaN,NaN,NaN,NaN,500,523,181.92,6.07,us-101,"(34.138592579539484, -118.36424630451197)"
8624055,515,2400,1123,1118848082000,29.749,339.730,6451311.529,1873143.649,13.0,6.9,...,NaN,NaN,NaN,NaN,500,523,165.14,6.92,us-101,"(34.13893551381322, -118.3646926491341)"


In [144]:
map101traf = folium.Map(location=[34.13923367949842, -118.365050201637], tiles="Stamen Toner", zoom_start=20)
map101traf

# Markers for frame appearances, click marker for mph.
appearance(1).apply(
    lambda row: folium.Marker(location=row["Coordinates"], popup=(row['v_Vel']/1.467)).add_to(map101traf), 
    axis=1)

map101traf

In [145]:
map101clear = folium.Map(location=[34.13923367949842, -118.365050201637], tiles="Stamen Toner", zoom_start=20)
map101clear

# Markers for frame appearances, click marker for mph.
appearance(15).apply(
    lambda row: folium.Marker(location=row["Coordinates"], popup=(row['v_Vel']/1.467)).add_to(map101clear), 
    axis=1)

map101clear